http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19GenAgeCaseInfJson?serviceKey=Iuowj7UW8R3d8WxSA%2Fh2wTsszZ1EZT22V%2FB%2BRBhNPW4bMCrxh0l89LSAmPjeCo1G%2FLjJTbmUbi%2FAM%2BL8Ae3cYQ%3D%3D&pageNo=1&numOfRows=10&startCreateDt=20200310&endCreateDt=20200414

In [122]:
import pandas as pd
from bs4 import BeautifulSoup
import requests 
from tqdm import tqdm
import numpy as np
from urllib.request import urlopen

In [124]:
key='Iuowj7UW8R3d8WxSA%2Fh2wTsszZ1EZT22V%2FB%2BRBhNPW4bMCrxh0l89LSAmPjeCo1G%2FLjJTbmUbi%2FAM%2BL8Ae3cYQ%3D%3D'
url= 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19GenAgeCaseInfJson?pageNo=1&numOfRows=10&startCreateDt=20200101&endCreateDt=20201030&serviceKey={}'.format(key)

page=requests.get(url)
soup=BeautifulSoup(page.content, "html.parser")

rows = soup.find_all('item')

rowList = []
nameList = []
columnList = []

rowsLen = len(rows)
for i in range(0, rowsLen):
    columns = rows[i].find_all()
    
    columnsLen = len(columns)
    for j in range(0, columnsLen):
        if i == 0:
            nameList.append(columns[j].name) 
        eachColumn = columns[j].text
        columnList.append(eachColumn)
    rowList.append(columnList)
    columnList = []    # 다음 row의 값을 넣기 위해 비워준다. (매우 중요!!)
    
result = pd.DataFrame(rowList, columns=nameList)
result.to_excel("age_gender.xlsx")

In [148]:
age_gen=pd.read_excel("age_gender.xlsx", header=1)
age_gen.rename(columns={'confcase':'확진자', 'confcaserate':'확진률', 'createdt':'등록일시분초',
                          'criticalrate':'치명률', 'death':'사망자', 'deathrate':'사망률','gubun':'구분(성별, 연령별)',
                          'seq':'확진자 성별, 연령별 고유값', 'updatedt':'수정일시분초'}, inplace=True)
age_gender=age_gender.sort_values(by= ['Unnamed: 0'], ascending=False)
age_gender

,Unnamed: 0,확진자,확진률,등록일시분초,치명률,사망자,사망률,"구분(성별, 연령별)","확진자 성별, 연령별 고유값",수정일시분초
2244,2244,3905,39.90,2020-04-02 19:02:48.48,2.10,82,50.62,남성,1,NaN
2243,2243,5881,60.10,2020-04-02 19:02:48.48,1.36,80,49.38,여성,2,NaN
2242,2242,442,4.52,2020-04-02 19:02:48.48,18.55,82,50.62,80 이상,3,NaN
2241,2241,651,6.65,2020-04-02 19:02:48.48,7.07,46,28.40,70-79,4,NaN
2240,2240,1235,12.62,2020-04-02 19:02:48.48,1.78,22,13.58,60-69,5,NaN
...,...,...,...,...,...,...,...,...,...,...
4,4,3545,13.44,2020-10-30 14:10:38.513,0.11,4,0.86,40-49,4332,NaN
3,3,3263,12.37,2020-10-30 14:10:38.513,0.06,2,0.43,30-39,4333,NaN
2,2,5147,19.51,2020-10-30 14:10:38.514,0.00,0,0.00,20-29,4334,NaN
1,1,1452,5.50,2020-10-30 14:10:38.514,0.00,0,0.00,10-19,4335,NaN


In [164]:
age_gender_t=age_gender[2234:2247]
age_gender_total=pd.pivot_table(age_gender_t, index=['구분(성별, 연령별)'],values=['확진률','사망자','사망률','치명률','확진자'])
age_gender_total

,사망률,사망자,치명률,확진률,확진자
"구분(성별, 연령별)",,,,,
0-9,0.00,0,0.00,2.60,685
10-19,0.00,0,0.00,5.50,1452
20-29,0.00,0,0.00,19.51,5147
30-39,0.43,2,0.06,12.37,3263
40-49,0.86,4,0.11,13.44,3545
50-59,4.54,21,0.43,18.36,4845
60-69,11.45,53,1.26,15.93,4202
70-79,32.83,152,7.22,7.98,2106
80 이상,49.89,231,20.26,4.32,1140
